# Best approach

Note: Lecture 7 is key to my problem

In [ ]:
import pandas as pd
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, pipeline

In [ ]:
df = load_from_disk("bld/python/data/data_clean")
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
model_name = "facebook/bart-large-mnli"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["Article text"], padding="max_length", truncation=True)

In [ ]:
df_encoded = df.map(tokenize, batched=True, batch_size=None)

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name,
    multi_label=True,
    device="cuda:0" if torch.cuda.is_available() else None,
)

In [ ]:
sequence_to_classify = (
    "Tiger Woods: Is this the end of his era? - CNN,Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. ",
    "golf, Tiger Woods: Is this the end of his era? - CNN,Is this the end of the Tiger Woods era?,This story was excerpted from the November 23 edition of CNN's Meanwhile in America, the daily email about US politics for global readers. Click here to read past editions and subscribe. (CNN)Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. Woods, who is recuperating from devastating leg injuries from a car crash, told Golf Digest he would have to be more selective about competition from now on. "
    "I think something that is realistic",
)

In [ ]:
classifier(sequence_to_classify, candidate_labels, tokenizer=tokenizer)

## Functionize it

#### Reasoning for new model

https://huggingface.co/valhalla/distilbart-mnli-12-1 has 90% of the facebook/bart-large-mnli model's performance but is way faster

In [ ]:
import pandas as pd
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, pipeline

In [ ]:
df = load_from_disk("bld/python/data/data_clean")
model_name = "facebook/bart-large-mnli"

In [ ]:
model_name_2 = "valhalla/distilbart-mnli-12-1"

In [ ]:
from transformers import AutoTokenizer


def zero_shot_labelling(data):
    model_name = "valhalla/distilbart-mnli-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16,  # adjust batch size
    )


# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"


def _tokenize(batch, tokenizer):
    return tokenizer(batch["Description"], padding=True, truncation=True, max_length=42)


# Call zero_shot_labelling function

In [ ]:
# automodel = AutoModelForSequenceClassification.from_pretrained(model_name)
from transformers import AutoTokenizer, AutoModel, , AutoModelForSequenceClassification


def zero_shot_labelling(data):
    model_name = "facebook/bart-large-mnli"
    tokenizer = AutoModelForSequenceClassification.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16, # adjust batch size
    )
# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"

def _tokenize(batch, tokenizer):
    return tokenizer(batch["Description"], padding="max_length", truncation=True)


# Call zero_shot_labelling function
df_encoded = zero_shot_labelling(df)

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name_2,
    multi_label=True,
    device="cuda:0" if torch.cuda.is_available() else None,
)

In [ ]:
sequence_to_classify = (
    "Tiger Woods: Is this the end of his era? - CNN,Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. ",
    "golf, Tiger Woods: Is this the end of his era? - CNN,Is this the end of the Tiger Woods era?,This story was excerpted from the November 23 edition of CNN's Meanwhile in America, the daily email about US politics for global readers. Click here to read past editions and subscribe. (CNN)Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. Woods, who is recuperating from devastating leg injuries from a car crash, told Golf Digest he would have to be more selective about competition from now on. "
    "I think something that is realistic",
)

In [ ]:
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
from datasets import Dataset, DatasetDict

df_try = df


def pd_to_dataset(data):
    data = Dataset.from_pandas(data)
    dataset_dict = DatasetDict({"my_dataset": data})
    return dataset_dict["my_dataset"]


df_encoded = zero_shot_labelling(df_try)

In [ ]:
df_encoded

In [ ]:
classifier(df_encoded["Description"], candidate_labels, tokenizer=_tokenize)

need to speed it up:
- Batch size of 8
- padding can be reduced to speed up computation
- 

## Approach to be faster

In [ ]:
from huggingface_hub import scan_cache_dir

delete_strategy = scan_cache_dir().delete_revisions(
    "81fd1d6e7847c99f5862c9fb81387956d99ec7aa"
    "e2983b237dccf3ab4937c97fa717319a9ca1a96d",
    "6c0e6080953db56375760c0471a8c5f2929baf11",
)
print("Will free " + delete_strategy.expected_freed_size_str)


delete_strategy.execute()

# Specify the directory you want to clear the cache for
cache_directory = "/path/to/your/cache/directory"

# Use scan_cache_dir to clear the cache in the specified directory
scan_cache_dir(cache_directory).clear()

# Neuer try

In [ ]:
import pandas as pd
import torch

In [ ]:
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, pipeline

df = load_from_disk("bld/python/data/data_clean")

In [ ]:
df = zero_shot_labelling(df)

In [ ]:
from transformers import AutoTokenizer


def zero_shot_labelling(data):
    model_name = "valhalla/distilbart-mnli-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16,  # adjust batch size
    )


# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"


def _tokenize(batch, tokenizer):
    return tokenizer(batch["Description"], padding=True, truncation=True, max_length=42)


# Call zero_shot_labelling function

In [ ]:
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
model_name_2 = "valhalla/distilbart-mnli-12-1"

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name_2,
    multi_label=True,
    device="cuda:0" if torch.cuda.is_available() else None,
)

In [ ]:
classifier(df["Description"], candidate_labels, tokenizer=_tokenize)

In [ ]:
df

## just functions

In [ ]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, pipeline

df = load_from_disk("bld/python/data/data_clean")

In [ ]:
first_100_entries = df.select(range(100))

In [ ]:
classif = zero_shot_classifier(first_100_entries)

In [ ]:
import random

import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, pipeline

seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


def zero_shot_classifier(data):
    """Classify the zero-shot data to receive the labels."""
    data = _zero_shot_labelling(data)
    model_name = "valhalla/distilbart-mnli-12-6"
    labels = ["labor supply", "labor demand", "government intervention"]
    classifier = pipeline(  # second last
        "zero-shot-classification",
        model=model_name,
        multi_label=True,
        device="cuda:0" if torch.cuda.is_available() else None,
    )
    return classifier(  # last
        data["Description"],
        labels,
        tokenizer=_tokenize,
    )


def _zero_shot_labelling(data):
    """Load the model for zero-shot classification and apply on the data."""
    model_name = "valhalla/distilbart-mnli-12-6"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=8,
    )


def _tokenize(batch, tokenizer):
    """Define the tokenizer."""
    return tokenizer(
        batch["Description"],
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

# For reading to know

In [ ]:
# run the previous
select_random_entries(df, num_entries=50, random_state=42)

# Training the model

TODO:
- does the probability of the model suits or should I transform to 0 and 1
- test and training separation
- model selection
- put the head on it

In [ ]:
from datasets import load_from_disk

df = load_from_disk(r"bld\python\labelled\data_labelled_subset")

In [ ]:
df

In [ ]:
data = create_dataset_dict_2(df)

In [ ]:
data.save_to_disk("bld/python/TrainTest")

In [ ]:
import datasets


def _split_dataset(df):
    # Shuffle the dataset to ensure randomization
    df = df.shuffle(seed=42)

    # Calculate the split sizes
    total_size = len(df)
    train_size = int(0.8 * total_size)
    val_size = int(0.1 * total_size)
    total_size - train_size - val_size

    # Split the dataset
    train_dataset = datasets.Dataset.from_dict(df[:train_size])
    val_dataset = datasets.Dataset.from_dict(df[train_size : train_size + val_size])
    test_dataset = datasets.Dataset.from_dict(df[train_size + val_size :])

    # Rename columns if needed
    train_dataset = train_dataset.rename_column("sequence", "text")
    val_dataset = val_dataset.rename_column("sequence", "text")
    test_dataset = test_dataset.rename_column("sequence", "text")

    # You may need to specify the 'labels' column name if it's different
    # Assuming it's 'labels' in your dataset, rename it to 'label'
    train_dataset = train_dataset.rename_column("labels", "label")
    val_dataset = val_dataset.rename_column("labels", "label")
    test_dataset = test_dataset.rename_column("labels", "label")

    return {
        "train_dataset": train_dataset,
        "val_dataset": val_dataset,
        "test_dataset": test_dataset,
    }


def create_dataset_dict_2(df):
    # Split the dataset using the split_dataset function
    split_data = _split_dataset(df)

    # Create a DatasetDict containing train, validation, and test datasets
    return datasets.DatasetDict(split_data)


# Example usage:
# df = Your existing dataset

# Multilabel Problem

In [ ]:
import pandas as pd
import torch
from datasets import load_from_disk
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    pipeline,
)

In [ ]:
df = load_from_disk("bld/python/TrainTest/TrainTest_data/")

In [ ]:
df["train_dataset"][0]

In [ ]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt,
    problem_type="multi_label_classification",
)

In [ ]:
df

In [ ]:
def tokenize_and_encode(df):
    return tokenizer(df["text"], truncation=True)

In [ ]:
cols = df["train_dataset"].column_names
cols

In [ ]:
cols = df["train_dataset"].column_names
cols.remove("scores")
df_enc = df.map(tokenize_and_encode, batched=True, remove_columns=cols)
df_enc

Scores and labels are badly named

In [ ]:
# cast label IDs to floats
import torch

df_enc.set_format("torch")
df_enc = df_enc.map(
    lambda x: {"float_labels": x["scores"].to(torch.float)},
    remove_columns=["scores"],
).rename_column("float_labels", "scores")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=1,
).to(device)

In [ ]:
df_enc

In [ ]:
args = TrainingArguments(".", num_train_epochs=1)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=df_enc["train_dataset"],
    eval_dataset=df_enc["val_dataset"],
    tokenizer=tokenizer,
)

In [ ]:
from torch import nn

In [ ]:
df_enc["train_dataset"]["input_ids"]

In [ ]:
# Convert input lists to tensors
input_ids = torch.tensor(df_enc["train_dataset"]["input_ids"])
attention_mask = torch.tensor(df_enc["train_dataset"]["attention_mask"])

# Forward pass through the model to get logits
logits = model(input_ids, attention_mask=attention_mask)

# Compute the loss
loss = criterion(
    logits.logits,
    scores.float(),
)  # Use logits.logits to access the raw logits

# Apply a threshold to the logits to determine class predictions (e.g., 0.5)
threshold = 0.5
predictions = (torch.sigmoid(logits.logits) >= threshold).int()

# 'predictions' now contains the predicted classes for each ex

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
df

In [ ]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=df["train_dataset"],
    eval_dataset=df["val_dataset"],
)

In [ ]:
trainer.train()

### additionall model/CURRENT ISSUE

In [ ]:
df = load_from_disk("bld/python/TrainTest/TrainTest_data/")

In [ ]:
df

In [ ]:
df["train_dataset"][0]

In [ ]:
import pandas as pd
import torch
from datasets import DatasetDict
from torch import nn
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

# Initialize a pre-trained tokenizer and model
model_name = "bert-base-uncased"  # You can change this to your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Assuming df_enc contains the dataset in the correct format
# df_enc should look like this:
# DatasetDict({
#     train_dataset: Dataset({
#         features: ['input_ids', 'attention_mask', 'scores'],
#     val_dataset: Dataset({
#         features: ['input_ids', 'attention_mask', 'scores'],
#     test_dataset: Dataset({
#         features: ['input_ids', 'attention_mask', 'scores'],

# Define the loss function for multi-label classification (e.g., BCEWithLogitsLoss)
criterion = nn.BCEWithLogitsLoss()

# Convert text data to input tensors using the tokenizer
df["train_dataset"] = tokenizer(
    df["train_dataset"]["text"],
    padding=True,
    truncation=True,
    return_tensors="pt",
)
df["val_dataset"] = tokenizer(
    df["val_dataset"]["text"],
    padding=True,
    truncation=True,
    return_tensors="pt",
)

# Forward pass through the model to get logits
input_ids = df["train_dataset"]["input_ids"]
attention_mask = df["train_dataset"]["attention_mask"]
logits = model(input_ids, attention_mask=attention_mask)

# Assuming 'scores' is already in the correct format
scores = torch.tensor(df["train_dataset"]["scores"], dtype=torch.float32)

# Compute the loss
loss = criterion(logits.logits, scores)

# Define your training arguments and trainer and train the model
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,  # Adjust as needed
    evaluation_strategy="steps",
    eval_steps=500,  # Adjust as needed
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # You can specify a data collator if needed
    train_dataset=df_enc["train_dataset"],  # Use your train_dataset here
    eval_dataset=df_enc["val_dataset"],  # Use your val_dataset here
)

trainer.train()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming df contains your dataset
labels = df["train_dataset"]["label"]

mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(labels)

In [ ]:
import torch
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    Trainer,
    TrainingArguments,
)

# Load the pre-trained model and tokenizer
model_name = "bert-base-cased"
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(df["train_dataset"]["scores"][0]),
)
tokenizer = BertTokenizer.from_pretrained(model_name)


# Function to preprocess the dataset and return it in the required format
def preprocess_function(examples):
    # Tokenize the text and encode it into input features
    inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
    )

    # Convert scores to a tensor (assuming scores are already in the correct format)
    scores = torch.tensor(examples["scores"], dtype=torch.float32)

    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": scores,
    }


# Preprocess the datasets
train_dataset = df["train_dataset"].map(preprocess_function)
val_dataset = df["val_dataset"].map(preprocess_function)
test_dataset = df["test_dataset"].map(preprocess_function)

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=2,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="tensorboard",
)


# Define a function to compute metrics
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = (predictions > 0).astype(int)  # Convert logits to binary predictions
    f1 = f1_score(labels, predictions, average="micro")
    precision = precision_score(labels, predictions, average="micro")
    recall = recall_score(labels, predictions, average="micro")
    return {"f1_micro": f1, "precision_micro": precision, "recall_micro": recall}


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Real Model

In [ ]:
import pandas as pd
import torch
from datasets import load_from_disk
from transformers import pipeline

In [ ]:
df = load_from_disk("bld/python/TrainTest/TrainTest_data/")

In [ ]:
df

In [ ]:
train_labels = df["train_dataset"].select_columns(["scores"])

In [ ]:
model_name = "bert-base-cased"

In [ ]:
df

In [ ]:
classifier = pipeline(
    "text-classification",
    model=model_name,
)

In [ ]:
# preprocess because values are apparently string and not int

model_output = pd.DataFrame(classifier(df["val_dataset"]["text"]))

In [ ]:
max_length = 512


def tokenize_data(df):
    return tokenizer(
        df["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
    )

In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)


df_encoded = df.map(tokenize, batched=True, batch_size=None)
df_encoded.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "label"],
)
df_encoded.set_format("torch")
# df_encoded

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
).to(device)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=5)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
df

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=df_encoded["train_dataset"],
    eval_dataset=df_encoded["val_dataset"],
)

In [ ]:
trainer.train()

## API Problem

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi


def authenticate_to_kaggle():
    """Authenticate to Kaggle."""
    api = KaggleApi()
    api.authenticate()
    return api

In [ ]:
task_load_data_python(path)

In [ ]:
path = "bld/python/data/"

In [ ]:
import zipfile


def task_load_data_python(path):
    """Clean the data (Python version).

    Download needs up to 5 minutes. Is this due to internet or coding issue?

    """
    api = authenticate_to_kaggle()
    dataset = "hadasu92/cnn-articles-after-basic-cleaning"
    api.dataset_download_files(dataset)
    with zipfile.ZipFile("cnn-articles-after-basic-cleaning.zip", "r") as zip_ref:
        zip_ref.extractall(path)